In [1]:
import numpy as np
import tensorflow as tf
import joblib

In [2]:
#구글 드라이브 import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
model_path = '/content/drive/MyDrive/Capstone1/제작 자료/TR_model'
scaler_path = '/content/drive/MyDrive/Capstone1/제작 자료/scaler.pkl'

model = tf.saved_model.load(model_path)
scaler = joblib.load(scaler_path)

# 서명 정보 확인
infer = model.signatures['serving_default']
print("Input Info:")
print(infer.structured_input_signature)
print("Output Info:")
print(infer.structured_outputs)

Input Info:
((), {'dense_input': TensorSpec(shape=(None, 2), dtype=tf.float32, name='dense_input')})
Output Info:
{'dense_5': TensorSpec(shape=(None, 5), dtype=tf.float32, name='dense_5')}


In [4]:
# 사용자가 입력하는 L과 Q 값을 받아옴
L = float(input("Enter the value for L: "))
Q = float(input("Enter the value for Q: "))

# 예측을 위한 입력 데이터 생성 및 정규화
input_data = np.array([[L, Q]], dtype=np.float32)
input_data = scaler.transform(input_data)

# 최적화 조건 확인 함수
def check_conditions(N, L1, L2, W, S):
    return (
        L1 >= 2 * N * W + (2 * N - 1) * S and
        L2 >= 2 * N * W + (2 * N - 1) * S and
        0.25 <= N <= 5 and
        2 <= L1 <= 25 and
        2 <= L2 <= 25 and
        0.05 <= W <= 1 and
        0.05 <= S <= 1
    )

# 최적화 조건을 만족하는 출력 찾기
found = False
max_attempts = 1000
for _ in range(max_attempts):
    # 예측 수행
    predictions = model(input_data, training=False).numpy()
    predicted_values = predictions[0]

    # N 값을 0.25 단위로 조정
    N = round(predicted_values[0] * 4) / 4
    L1, L2, W, S = np.round(predicted_values[1:], 2)
    if check_conditions(N, L1, L2, W, S):
        found = True
        break

if found:
    print("Optimal Predictions found:", [N, L1, L2, W, S])
else:
    print("No optimal predictions found within the maximum attempts.")


Enter the value for L: 1000
Enter the value for Q: 14
Optimal Predictions found: [2.0, 14.16, 11.19, 0.15, 0.3]
